<a href="https://colab.research.google.com/github/muralikrishnat29/object-detection-keras-retinanet/blob/master/keras_to_tfpbtrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import shutil 
import os

In [3]:
%cd /content/
!pip install keras -U

/content
     |████████████████████████████████| 378kB 5.0MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
# Helper For getting the model Size
import os
from tensorflow.python import ops
def get_size(model_dir, model_file='saved_model.pb'):
  model_file_path = os.path.join(model_dir, model_file)
  print(model_file_path, '')
  pb_size = os.path.getsize(model_file_path)
  variables_size = 0
  if os.path.exists(
      os.path.join(model_dir,'variables/variables.data-00000-of-00001')):
    variables_size = os.path.getsize(os.path.join(
        model_dir,'variables/variables.data-00000-of-00001'))
    variables_size += os.path.getsize(os.path.join(
        model_dir,'variables/variables.index'))
  print('Model size: {} KB'.format(round(pb_size/(1024.0),3)))
  print('Variables size: {} KB'.format(round( variables_size/(1024.0),3)))
  print('Total Size: {} KB'.format(round((pb_size + variables_size)/(1024.0),3)))

In [0]:
# To Freeze the Saved Model
# We need to freeze the model to do further optimisation on it

from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools import freeze_graph
from tensorflow.python import ops
from tensorflow.tools.graph_transforms import TransformGraph

def freeze_model(saved_model_dir, output_node_names, output_filename):
  output_graph_filename = os.path.join(saved_model_dir, output_filename)
  initializer_nodes = ''
  freeze_graph.freeze_graph(
      input_saved_model_dir=saved_model_dir,
      output_graph=output_graph_filename,
      saved_model_tags = tag_constants.SERVING,
      output_node_names=output_node_names,
      initializer_nodes=initializer_nodes,
      input_graph=None,
      input_saver=False,
      input_binary=False,
      input_checkpoint=None,
      restore_op_name=None,
      filename_tensor_name=None,
      clear_devices=True,
      input_meta_graph=False,
  )

In [0]:
# Source https://medium.com/google-cloud/optimizing-tensorflow-models-for-serving-959080e9ddbf
# Convert the frozen graph to TF model

# Note -You need to change the outputs for your model
# Helper for Converting Frozen graph from Disk to TF serving compatible Model
def get_graph_def_from_file(graph_filepath):
  tf.reset_default_graph()
  with ops.Graph().as_default():
    with tf.gfile.GFile(graph_filepath, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      return graph_def

def convert_graph_def_to_saved_model(export_dir, graph_filepath):

  graph_def = get_graph_def_from_file(graph_filepath)
  with tf.Session(graph=tf.Graph()) as session:
    tf.import_graph_def(graph_def, name='')
    tf.saved_model.simple_save(
        session,
        export_dir,
        inputs={'input_image': session.graph.get_tensor_by_name('{}:0'.format(node.name))
            for node in graph_def.node if node.op=='Placeholder'},
        outputs={t:session.graph.get_tensor_by_name(t) for t in outputs}
                        
                
    )
    print('Optimized graph converted to SavedModel!')

In [0]:
# Optimizing the graph via TensorFlow library
from tensorflow.tools.graph_transforms import TransformGraph


def optimize_graph(model_dir, graph_filename, transforms, output_names, outname='optimized_model.pb'):
  input_names = ['input_image',] # change this as per how you have saved the model
  graph_def = get_graph_def_from_file(os.path.join(model_dir, graph_filename))
  optimized_graph_def = TransformGraph(
      graph_def,
      input_names,  
      output_names,
      transforms)
  tf.train.write_graph(optimized_graph_def,
                      logdir=model_dir,
                      as_text=False,
                      name=outname)
  print('Graph optimized!')

In [9]:
#Install keras-retinanet
#Do this  only once
!git clone --recursive https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 5596 (delta 19), reused 21 (delta 9), pack-reused 5557
Receiving objects: 100% (5596/5596), 14.76 MiB | 26.65 MiB/s, done.
Resolving deltas: 100% (3737/3737), done.
Submodule 'tests/test-data' (https://github.com/fizyr/keras-retinanet-test-data.git) registered for path 'tests/test-data'
Cloning into '/content/keras-retinanet/tests/test-data'...
remote: Enumerating objects: 9, done.        
remote: Counting objects: 100% (9/9), done.        
remote: Compressing objects: 100% (6/6), done.        
remote: Total 45 (delta 1), reused 6 (delta 1), pack-reused 36        
Submodule path 'tests/test-data': checked out '98404379fbf1ff1273d01db835c10cc83a4f8007'


In [10]:
!cd keras-retinanet && git pull

Already up to date.


In [11]:
#Do this only once
%cd keras-retinanet/
!pip install . -q
!python setup.py build_ext --inplace

/content/keras-retinanet
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_ret

In [0]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = '/content/drive/My Drive/detection_models/resnet50_csv_11.h5'

In [13]:
# load retinanet model
from keras_retinanet import models
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'traffic-light'}

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


Saving the Keras Model in TF Serving compatible format


In [14]:
import shutil 
import os
import keras
import tensorflow as tf
from keras.models import load_model


#session = tf.keras.backend.get_session()
session =keras.backend.tensorflow_backend.get_session()

tf.keras.backend.set_learning_phase(0) # Ignore dropout at inference

print("Model file path",model_path)
weightfile = model_path
export_path = os.path.join('keras-retinanet')


if os.path.exists( export_path+"/4"):
    shutil.rmtree( export_path+"/4")
    
export_path = export_path+"/4"

#with tf.keras.backend.get_session() as sess: --> this wont work
with session as sess:
    tf.saved_model.simple_save(
                session,
                export_path,
                inputs={'input_image': model.input},
                outputs={t.name:t for t in model.outputs})

Model file path /content/drive/My Drive/detection_models/resnet50_csv_11.h5
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/4/saved_model.pb


In [15]:
!saved_model_cli show --dir '/content/keras-retinanet/keras-retinanet/4/' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, -1, -1, 3)
        name: input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 300, 4)
        name: filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0
    outputs['filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 300)
        name: filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0
    outputs['filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 300)
        name: filtered_detections/map/TensorArraySt

Convert the Saved model for Freezing (for Optimizations)

In [17]:
# Try to load the saved model
%mkdir /content/keras-retinanet/keras-retinanet/5
tf.keras.backend.clear_session()
session = tf.Session()
import_path = os.path.join('keras-retinanet/4')
        
meta =tf.saved_model.loader.load(
            session,
            [tf.saved_model.tag_constants.SERVING],
        import_path )

output_node_names = u'filtered_detections/map/TensorArrayStack/TensorArrayGatherV3,\
filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3,     \
filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3'

freeze_model('keras-retinanet/4/', output_node_names, '../5/freezed_model.pb')

INFO:tensorflow:Restoring parameters from keras-retinanet/4/variables/variables
INFO:tensorflow:Restoring parameters from keras-retinanet/4/variables/variables
INFO:tensorflow:Froze 306 variables.
INFO:tensorflow:Converted 306 variables to const ops.


In [19]:
get_size('keras-retinanet/4/',model_file='saved_model.pb')
#print "------------------------------------"
get_size('keras-retinanet/5/',model_file='freezed_model.pb')
#print "-------------------------------------"
#print "saved frozen model"
get_size('keras-retinanet/4/',model_file='saved_model.pb')

keras-retinanet/4/saved_model.pb 
Model size: 1670.117 KB
Variables size: 243036.61 KB
Total Size: 244706.728 KB
keras-retinanet/5/freezed_model.pb 
Model size: 142680.328 KB
Variables size: 0.0 KB
Total Size: 142680.328 KB
keras-retinanet/4/saved_model.pb 
Model size: 1670.117 KB
Variables size: 243036.61 KB
Total Size: 244706.728 KB


``Convert Frozen model to TF Serving

In [0]:
# You need to change the outputs for your model
outputs = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',\
'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',     \
'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

Load a frozen model

In [21]:
#Lets load a frozen model
gdef =get_graph_def_from_file('keras-retinanet/5/freezed_model.pb')
convert_graph_def_to_saved_model('keras-retinanet/xx/', 'keras-retinanet/5/freezed_model.pb')

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/xx/saved_model.pb
Optimized graph converted to SavedModel!


In [22]:
# Helper Run this to get the node names in the graph
# useful to get the node names to be used in framing the output 
node_names = [node.name for node in gdef.node]
print(node_names)

['input_1', 'pyramid_regression_0/kernel', 'pyramid_regression_0/bias', 'pyramid_regression_1/kernel', 'pyramid_regression_1/bias', 'pyramid_regression_2/kernel', 'pyramid_regression_2/bias', 'pyramid_regression_3/kernel', 'pyramid_regression_3/bias', 'pyramid_regression/kernel', 'pyramid_regression/bias', 'pyramid_classification_0/kernel', 'pyramid_classification_0/bias', 'pyramid_classification_1/kernel', 'pyramid_classification_1/bias', 'pyramid_classification_2/kernel', 'pyramid_classification_2/bias', 'pyramid_classification_3/kernel', 'pyramid_classification_3/bias', 'pyramid_classification/kernel', 'pyramid_classification/bias', 'padding_conv1/Pad/paddings', 'padding_conv1/Pad', 'conv1/kernel', 'conv1/convolution/ReadVariableOp', 'conv1/convolution', 'bn_conv1/gamma', 'bn_conv1/beta', 'bn_conv1/moving_mean', 'bn_conv1/moving_variance', 'bn_conv1/ReadVariableOp', 'bn_conv1/ReadVariableOp_1', 'bn_conv1/FusedBatchNormV3/ReadVariableOp', 'bn_conv1/FusedBatchNormV3/ReadVariableOp_1',

Optimising the Frozen model

In [23]:
# Optimization without Qunatization
transforms = ['remove_nodes(op=Identity)', \
 'merge_duplicate_nodes', \
 'strip_unused_nodes',
 'fold_constants(ignore_errors=true)',
 'fold_batch_norms',
 'quantize_weights'] #this reduces the size, but there is no speed up , actaully slows down, see below
output_node_names = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3',\
'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3',     
'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3']

optimize_graph('keras-retinanet/5/', 'freezed_model.pb' , transforms, output_node_names,outname='optimized_model_simple.pb')

Graph optimized!


In [24]:
# see https://medium.com/google-cloud/optimizing-tensorflow-models-for-serving-959080e9ddbf
print("Optimised models")
get_size('keras-retinanet/5/',model_file='optimized_model_simple.pb') 

# Only weights are quanized - not faster actually slower though size has decreased to just 37 MB from 149 MB!!
# Also accuracy seems also to be similar
#('Time for ', 10, ' is ', 0.6285901069641113) Non optimized v100
#('Time for ', 10, ' is ', 0.6342859268188477) - after optimization but without quanization
#('Time for ', 10, ' is ', 1.1140730381011963) After weight quantization  - time became almost doub;e

#"This process converts all the operations in the graph that have eight-bit quantized equivalents, 
#and leaves the rest in floating point. Only a subset of ops are supported, and on many platforms 
#the quantized code may actually be slower than the float equivalents, 
#but this is a way of increasing performance substantially when all the circumstances are right.
# from https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms#optimizing-for-deployment


Optimised models
keras-retinanet/5/optimized_model_simple.pb 
Model size: 35982.76 KB
Variables size: 0.0 KB
Total Size: 35982.76 KB


In [0]:
# You need to change the outputs for your model
outputs = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',\
'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',     \
'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

In [26]:
%mkdir /content/keras-retinanet/keras-retinanet/6
export_path = os.path.join('keras-retinanet')
if os.path.exists( export_path+"/6"):
    shutil.rmtree( export_path+"/6")
convert_graph_def_to_saved_model('keras-retinanet/6/', 'keras-retinanet/5/optimized_model_simple.pb')

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/6/saved_model.pb
Optimized graph converted to SavedModel!


Optimise by tensorRT

In [27]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# You need to change the outputs for your model
outputs = ['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',\
'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',     \
'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0']

In [29]:
# Inference with TF-TRT `SavedModel` workflow:
%mkdir /content/keras-retinanet/keras-retinanet/inference_tensorRT
export_path = os.path.join('keras-retinanet')
if os.path.exists( export_path+"/inference_tensorRT"):
    shutil.rmtree( export_path+"/inference_tensorRT")
    
export_path = export_path+"/inference_tensorRT"


tf.reset_default_graph()
graph = tf.Graph()
sess = tf.Session()
# Create a TensorRT inference graph from a SavedModel:
with graph.as_default():
    with tf.Session() as sess:
        trt_graph = trt.create_inference_graph(
            input_graph_def=None,
            outputs=outputs,
            input_saved_model_dir='keras-retinanet/4/',
            input_saved_model_tags=['serve'],
            max_batch_size=1,
            max_workspace_size_bytes=7000000000,
            #precision_mode='FP16') 
            precision_mode='FP32')
            #precision_mode='INT8')
        output_node=tf.import_graph_def(trt_graph, return_elements=outputs)
        #sess.run(output_node)
        tf.saved_model.simple_save(sess,
        export_path,
        inputs={'input_image': graph.get_tensor_by_name('{}:0'.format(node.name))
            for node in graph.as_graph_def().node if node.op=='Placeholder'},
        outputs={t:graph.get_tensor_by_name('import/'+t) for t in outputs}
    
    )

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
INFO:tensorflow:Restoring parameters from keras-retinanet/4/variables/variables
INFO:tensorflow:Froze 306 variables.
INFO:tensorflow:Converted 306 variables to const ops.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/inference_tensorRT/saved_model.pb


In [31]:
!saved_model_cli show --dir 'keras-retinanet/inference_tensorRT/' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_image'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, -1, -1, 3)
        name: import/input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_FLOAT
        shape: unknown_rank
        name: import/filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0
    outputs['filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_FLOAT
        shape: unknown_rank
        name: import/filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0
    outputs['filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0'] tensor_info:
        dtype: DT_INT32
        shape: unknown_rank
        name: import/fil

In [31]:
# Inference with TF-TRT `Frozen` workflow:
# https://medium.com/tensorflow/speed-up-tensorflow-inference-on-gpus-with-tensorrt-13b49f3db3fa
%mkdir /content/keras-retinanet/keras-retinanet/inference_tensorRT1
export_path = os.path.join('keras-retinanet')
if os.path.exists( export_path+"/inference_tensorRT1"):
    shutil.rmtree( export_path+"/inference_tensorRT1")
    
export_path = export_path+"/inference_tensorRT1"

#Lets load a frozen model
gdef =get_graph_def_from_file('keras-retinanet/5/freezed_model.pb')

tf.reset_default_graph()
graph = tf.Graph()
sess = tf.Session()
# Create a TensorRT inference graph from a SavedModel:
with graph.as_default():
    with tf.Session() as sess:
        trt_graph = trt.create_inference_graph(
            input_graph_def=gdef,
            outputs=outputs,
            max_batch_size=1,
            max_workspace_size_bytes=7000000000,
            precision_mode='FP16') 
            #precision_mode='FP32')
            #precision_mode='INT8')
        output_node=tf.import_graph_def(trt_graph, return_elements=outputs)
        #sess.run(output_node)
        tf.saved_model.simple_save(sess,
        export_path,
        inputs={'input_image': graph.get_tensor_by_name('{}:0'.format(node.name))
            for node in graph.as_graph_def().node if node.op=='Placeholder'},
        outputs={t:graph.get_tensor_by_name('import/'+t) for t in outputs}
    
    )

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/inference_tensorRT1/saved_model.pb


In [33]:
# Inference with TF-TRT `Frozen` workflow:
# https://medium.com/tensorflow/speed-up-tensorflow-inference-on-gpus-with-tensorrt-13b49f3db3fa
%mkdir /content/keras-retinanet/keras-retinanet/inference_tensorRT2
export_path = os.path.join('keras-retinanet')
if os.path.exists( export_path+"/inference_tensorRT2"):
    shutil.rmtree( export_path+"/inference_tensorRT2")
    
export_path = export_path+"/inference_tensorRT2"

#Lets load a frozen model
gdef =get_graph_def_from_file('keras-retinanet/5/freezed_model.pb')

tf.reset_default_graph()
graph = tf.Graph()
sess = tf.Session()
# Create a TensorRT inference graph from a SavedModel:
with graph.as_default():
    with tf.Session() as sess:
        trt_graph = trt.create_inference_graph(
            input_graph_def=gdef,
            outputs=outputs,
            max_batch_size=1,
            max_workspace_size_bytes=7000000000,
            #precision_mode='FP16') 
            #precision_mode='FP32')
            precision_mode='INT8')
        output_node=tf.import_graph_def(trt_graph, return_elements=outputs)
        #sess.run(output_node)
        tf.saved_model.simple_save(sess,
        export_path,
        inputs={'input_image': graph.get_tensor_by_name('{}:0'.format(node.name))
            for node in graph.as_graph_def().node if node.op=='Placeholder'},
        outputs={t:graph.get_tensor_by_name('import/'+t) for t in outputs}
    
    )

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: keras-retinanet/inference_tensorRT2/saved_model.pb


In [34]:
print("---------NVIDIA RT Optimised model FP32")
get_size('keras-retinanet/inference_tensorRT/',model_file='saved_model.pb') #also nodes quantized
print("---------NVIDIA RT Optimised model FP16")
get_size('keras-retinanet/inference_tensorRT1/',model_file='saved_model.pb') #also nodes quantized
print("-------NVIDIA RT Optimised model INT8")
get_size('keras-retinanet/inference_tensorRT2/',model_file='saved_model.pb') #also nodes quantized

---------NVIDIA RT Optimised model FP32
keras-retinanet/inference_tensorRT/saved_model.pb 
Model size: 219049.253 KB
Variables size: 0.0 KB
Total Size: 219049.253 KB
---------NVIDIA RT Optimised model FP16
keras-retinanet/inference_tensorRT1/saved_model.pb 
Model size: 219049.253 KB
Variables size: 0.0 KB
Total Size: 219049.253 KB
-------NVIDIA RT Optimised model INT8
keras-retinanet/inference_tensorRT2/saved_model.pb 
Model size: 219049.253 KB
Variables size: 0.0 KB
Total Size: 219049.253 KB


In [35]:
%mkdir /content/drive/My\ Drive/detection_models/inference
%mkdir /content/drive/My\ Drive/detection_models/inference/tensorRT1
%mkdir /content/drive/My\ Drive/detection_models/inference/tensorRT2
%mkdir /content/drive/My\ Drive/detection_models/inference/tensorRT3
%mv -v /content/keras-retinanet/keras-retinanet/inference_tensorRT/* /content/drive/My\ Drive/detection_models/inference/tensorRT1/
%mv -v /content/keras-retinanet/keras-retinanet/inference_tensorRT1/* /content/drive/My\ Drive/detection_models/inference/tensorRT2/
%mv -v /content/keras-retinanet/keras-retinanet/inference_tensorRT2/* /content/drive/My\ Drive/detection_models/inference/tensorRT3/

mkdir: cannot create directory ‘/content/drive/My Drive/detection_models/inference’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/detection_models/inference/tensorRT1’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/detection_models/inference/tensorRT2’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/detection_models/inference/tensorRT3’: File exists
copied '/content/keras-retinanet/keras-retinanet/inference_tensorRT/saved_model.pb' -> '/content/drive/My Drive/detection_models/inference/tensorRT1/saved_model.pb'
removed '/content/keras-retinanet/keras-retinanet/inference_tensorRT/saved_model.pb'
created directory '/content/drive/My Drive/detection_models/inference/tensorRT1/variables'
removed directory '/content/keras-retinanet/keras-retinanet/inference_tensorRT/variables'
copied '/content/keras-retinanet/keras-retinanet/inference_tensorRT1/saved_model.pb' -> '/content/drive/My Drive/detection_models/inference/tensorRT2/s